In [1]:
from message import Message
from schema_registry import SchemaRClient
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry.avro import AvroSerializer

config = config = {
    'bootstrap.servers': 'pkc-12576z.us-west2.gcp.confluent.cloud:9092',
    'sasl.username':     '',
    'sasl.password':     '',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms':   'PLAIN',
    'acks':              'all'
}

producer = Producer(config)


In [2]:
def convert_to_dict(message,ctx):
    return dict(
        idade=message.idade,
        name=message.name,
        team=message.team)


def delivery_callback(err, msg):
    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))

In [3]:
scr_client = SchemaRClient("https://@psrc-57o65q.us-west2.gcp.confluent.cloud",
                                       "",
                                       ""
                                       )

schema_str = scr_client.get_schema_str(100001)
avro_serializer = AvroSerializer(scr_client.client,
                                     schema_str,
                                     convert_to_dict)

string_serializer = StringSerializer('utf_8')

In [5]:
topic = "topic_0"
from uuid import uuid4

message  = Message(20,"Edney","ML")
producer.produce(topic=topic,
                             key=string_serializer(str(uuid4())),
                             value=avro_serializer(message, SerializationContext(topic, MessageField.VALUE)),
                             on_delivery=delivery_callback)
producer.poll(10000)
producer.flush()


User record b'e114abb1-1e65-42b2-ace3-138fa2d9215c' successfully produced to topic_0 [0] at offset 13


0